**Auteur**: Voahangy Joan Aléonard<br>
**Dernière mise à jour** : 21 juillet 2021

# <font color=red>**Déploiement de l'Azure Functions en production**</font>

# <font color=green>**L'environnement Serverless d'Azure Functions**</font>

## <font color=blue>**Configuration de l'environnement de développement**</font>

---

Il faut disposer :
- d'un [compte Azure](https://azure.microsoft.com/fr-fr/) ;
- d'[Azure Functions Core Tools](https://www.npmjs.com/package/azure-functions-core-tools), qui fournit une expérience de développement locale pour créer, développer, tester, exécuter et déboguer Azure Functions;
- une version de Python prise en charge par Azure Functions : en juillet 2021, disponibilité générale (Python 3.8, 3.7 et 3.6) - préversion (Python 3.9) ;
- [Visual Studio Code](https://code.visualstudio.com/) ;
- [Extension Python](https://marketplace.visualstudio.com/items?itemName=ms-python.python) pour Visual Studio Code ;
- [Extension Azure Functions](https://marketplace.visualstudio.com/items?itemName=ms-azuretools.vscode-azurefunctions) pour Visual Studio Code ;
- [Microsoft Azure Storage Explorer](https://azure.microsoft.com/fr-fr/features/storage-explorer/), une interface graphique utilisateur (GUI) intuitive pour la gestion complète des ressources de stockage Cloud.

## <font color=blue>**Présentation du plan d'hébergement**</font>

---

Pour héberger notre fonction, nous avons choisi l'option d'**hébergement par défaut** qui est **Consumption Plan** (à la Consommation vs. Plan Premium ou Plan dédié (*App Service*)). 

***Note***: Linux est le seul système d’exploitation pris en charge pour la pile d’exécution Python, quel que soit le Plan choisi.

***Avantages:***
- On est facturé uniquement lorsque les fonctions sont en cours d'exécution ;
- La mise à l'échelle est automatique, même pendant les pics de charges, mais également jusqu'à zéro (inactivité).

***Inconvénients:***
- Les requêtes peuvent présenter une latence au démarrage (après une inactivité); c'est le démarrage à froid.

***Limites du services:***
- Durée du délai d'expiration (*timeout*) en minutes : 5 par défaut, 10 au maximum ; il faut noter que, quel que soit le paramètre de délai d’expiration du conteneur de fonctions, 230 secondes est le temps maximum que peut prendre une fonction déclenchée via HTTP pour répondre à une demande ;
- Applications de fonction par plan : 100 ;
- <font color=red>Mémoire maximale (en Go par instance de fonction) : 1.5</font> ;
- Nombre maximal d'instances : 200 ; l’infrastructure Azure Functions met automatiquement à l’échelle les ressources de processeur et de mémoire en ajoutant des instances de l’hôte Functions selon le nombre d’événements déclencheurs entrants.

***Facturation:***<br>
La facturation est basée sur le nombre d'exécutions, sur la durée d'exécution et la mémoire utilisée (*inclus: allocation mensuelle gratuite de 1 millions de requêtes exécutées et 400.000 secondes par Go de consommation de ressources*).
- 0,000014 €/secondes par Go pour le **délai d'exécution**;
- 0,169 € par million d’exécutions, avec un comptage du **nombre total d'exécutions** par mois pour l'ensemble des fonctions exécutées en réponse à un évènement (*event-driven*) qui est déclenché (*trigger*) par une liaison (*binding*).

# <font color=green>**Création des fichiers allégés**</font>

---

<font color=red>Etant limité à 1.5Go de mémoire</font>, il est nécessaire de "limiter" la taille des fichiers chargés dans le conteneur du Web Blob Storage.

Ce notebook est dédié à cet objectif.

In [1]:
# Import libraries
import pandas as pd
import numpy as np
from pathlib import Path
import pickle
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# Set data path
src_path = Path('/content/drive/MyDrive/OC_IA/P09/data')

## <font color=blue>**Données de clics**</font>

Comme nous avons 10.000 utilisateurs sur notre application, nous allons limiter nos données à ce nombre.

In [3]:
# Load data
clicks = pd.read_csv(src_path / 'clicks.csv')

# Create new file with user_id filtering (10000)
small_clicks = clicks[clicks.user_id.isin(range(0,10000))]

# Display shape and 5 first rows
print(small_clicks.shape)
small_clicks.head()

(225648, 6)


,user_id,session_id,session_start,session_size,article_id,click_timestamp
0,0,1506825423271737,1506825423000,2,157541,1506826828020
1,0,1506825423271737,1506825423000,2,68866,1506826858020
2,1,1506825426267738,1506825426000,2,235840,1506827017951
3,1,1506825426267738,1506825426000,2,96663,1506827047951
4,2,1506825435299739,1506825435000,2,119592,1506827090575


In [4]:
# Nb of unique article_ID
small_clicks.article_id.nunique()

12423

In [5]:
# Save the small clicks data to CSV
small_clicks.to_csv(src_path / 'small_clicks.csv', index=False)

## <font color=blue>**Données d'embeddings**</font>

On garde uniquement les articles consultés par les 10000 utilisateurs, soit 12.423.

L'autre alternative serait de récupérer d'autres articles (par exemple 20% en plus). Nous n'avons pas retenu cette solution pour ne pas alourdir le fichier, mais le script associé est disponible.

In [6]:
# Load data and convert to DF
embeddings = pd.read_pickle(src_path / 'articles_embeddings.pickle')
df_embeddings = pd.DataFrame(embeddings)

# Build the article IDs list
list_articleID = sorted(list(small_clicks.article_id.unique()))

# Create new DF with remaining article_ID list
# other_clicks = clicks[~clicks.article_id.isin(list_articleID)].sample(frac=.20)
# print(other_clicks.shape)

# Create a TOTAL list == 27879 article IDs
# total_articleID = sorted(list(list_articleID) + other_clicks.article_id.to_list())
# print(len(total_articleID))

# Create a filtered DF
test_embeddings = df_embeddings[df_embeddings.index.isin(list_articleID)]

# Convert to numpy
small_embeddings = test_embeddings.to_numpy()
small_embeddings.shape

(12423, 250)

In [7]:
# Save as Pickle
pickle.dump(small_embeddings, open(src_path / 'small_embeddings.pickle', 'wb'))

# Reload if necessary
small_embeddings = pd.read_pickle(src_path / 'small_embeddings.pickle')

# <font color=green>**Adaptation du script du modèle (*content-based*)**</font>

---

Nous adaptons notre script pour qu'il puisse :
- d'une part, faire pointer correctement un articleID à l'embeddings correspondant (en effet, un Numpy array n'a pas d'index explicite!);
- et d'autre part, afficher la liste des recommandations dans un format qui peut être lu par notre application mobile.

## <font color=blue>**Fonction de recommandations**</font>

---

La fonction retourne une liste de 5 articles recommendés.

In [8]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def get_ContentBased_Reco(userID, small_clicks, small_embeddings, n_reco=5):
    '''Return 5 recommended articles ID to user'''

    # Get the list of articles viewed by the user
    var = small_clicks.loc[small_clicks.user_id == userID]['article_id'].to_list()

    # Get the list of unique article_ID in small_clicks
    list_articleID = sorted(list(small_clicks.article_id.unique()))

    # Retrieve the corresponding index of the articles viewed by userID in var
    idx_var = []
    for i in range(0, len(var)):
        for idx, item in enumerate(list(list_articleID)):
            if item == var[i]:
                idx_var.append(idx)

    # Select the last element of the list
    value = idx_var[-1]
    # print(value)

    # Compute the cosine similarity
    emb = small_embeddings
    distances = cosine_similarity([emb[value]], emb)[0]

    # Save the result in Pandas DataFrame
    df_reco = pd.DataFrame(list(zip(list_articleID, distances)),
                           columns=(["reco_article_id", "similarity"]))
    
    # Sort the DF by similarity
    df_reco.sort_values(by='similarity', ascending=False, inplace=True)

    # Exclude already viewed articles
    top_reco = df_reco[~df_reco.reco_article_id.isin(var)]

    # Give the list of recommended articles
    result = list(top_reco["reco_article_id"].iloc[:(n_reco)].values)

    return result

In [9]:
#### Test the function ####
# Choose a userID
userID = 9332

# Save the function in a variable
reco5 = get_ContentBased_Reco(userID, small_clicks, small_embeddings, n_reco=5)

# Display the result
reco5

[162605, 157725, 158040, 161868, 158657]

## <font color=blue>**Script additionnel dans <code>__init__.py</code> >> <code>main</code>**</font>

---

C'est la partie que nous allons rajouté dans la fonction <code>main</code> pour permettre à l'application mobile d'afficher correctement les recommandations, en coordination avec la correction du fichier <code>App.js</code>.

In [10]:
# Convert as STRING
str_result = ' '.join(str(elem)+"," for elem in reco5)
print(str_result)

162605, 157725, 158040, 161868, 158657,


In [11]:
# Delete the last comma
result = str_result.rstrip(str_result[-1])
print(result)

162605, 157725, 158040, 161868, 158657


# <font color=green>**De la création du projet de fonction en local jusqu'au déploiement en production**</font>

## <font color=blue>**Tutorial Azure Functions avec Visual Studio Code**</font>

---

Un tutoriel de création, de test et de mise en production d'une Azure Functions est disponible à ce [lien](https://docs.microsoft.com/fr-fr/azure/azure-functions/create-first-function-vs-code-python).

Un [mode opératoire](https://s3.eu-west-1.amazonaws.com/course.oc-static.com/projects/Ing%C3%A9nieur_IA_P9/Mode+ope%CC%81ratoire+test+Azure+function_V1.1.docx.pdf) a également été fourni pour ce projet

**Les étapes globales sont les suivantes** :
<font color=blue>
1. Création d'un projet en local ;
2. Exécution et déboggage de la fonction localement ;
3. Publication/Déploiement du projet sur Azure ;
4. Exécution de la fonction dans Azure ;
5. Suppression/Nettoyage du Groupe de ressources lorsque l'application n'est plus utilisée.

## <font color=blue>**Finalisation après 1er déploiement**</font>

---
**Des étapes supplémentaires ont été effectuées apères la publication du projet** :
<font color=blue>
1. Création d'un container de Blob nommé *data* ;
2. Chargement des fichiers (*Blob*) allégés de **clicks** et d'**embeddings** ;
3. Création des liaisons d'entrée afin de spécifier les fichiers comme point d'entrée (mise à jour de <code>function.json</code>) ;
4. Mise à jour du Cross-Origin Resource Sharing (CORS) en autorisant toutes origines avec <code>*</code> ;
5. Correction du fichier <code>app.js</code> de **Bookshelf** ;
6. Adaptation du code <code>init.py</code> ;
7. Redéploiement et test de l'application.

### **function.json**

---

Note: Il faut mettre le nom du conteneur **data** sur le path.

```
{
  "scriptFile": "__init__.py",
  "bindings": [
    {
      "authLevel": "anonymous",
      "type": "httpTrigger",
      "direction": "in",
      "name": "req",
      "methods": [
        "get",
        "post"
      ]
    },
    {
      "type": "http",
      "direction": "out",
      "name": "$return"
    },
    {
      "name": "clicksBlob",
      "type": "blob",
      "dataType": "binary",
      "path": "data/small_clicks.csv",
      "connection": "AzureWebJobsStorage",
      "direction": "in"
    },
    {
      "name": "embeddingsBlob",
      "type": "blob",
      "dataType": "binary",
      "path": "data/small_embeddings.pickle",
      "connection": "AzureWebJobsStorage",
      "direction": "in"
    }

  ]
}

```



### **__init__.py**

---

```
# Use Azure main function to get the recommendations
def main(req: func.HttpRequest, clicksBlob: func.InputStream, embeddingsBlob: func.InputStream) -> func.HttpResponse:
    logging.info('Python HTTP trigger function processed a request.')

    # Load the data from AzureBlobStorage
    clicks = pd.read_csv(BytesIO(clicksBlob.read()), index_col=None, header=0)
    print('click: ', clicks.shape)
    embeddings = pd.read_pickle(BytesIO(embeddingsBlob.read()))
    print('emb ', embeddings.shape)
    
    # Get the userID
    #!!! The parameter from bookshelf is 'userId' !!!#
    userID = req.params.get('userId')
    if not userID:
        try:
            req_body = req.get_json()
        except ValueError:
            pass
        else:
            userID = req_body.get('userId')

    if userID:
        # Get the 5 articles' recommendations
        userID = int(userID)
        reco5 = get_ContentBased_Reco(int(userID), clicks, embeddings, n_reco=5)

        str_result = ' '.join(str(elem)+"," for elem in reco5)
        result = str_result.rstrip(str_result[-1])

        # Template example is to return a sentence with the user_id
        return func.HttpResponse(result)

    else:
        return func.HttpResponse(
             "This HTTP triggered function executed successfully. Please enter a userID.", status_code=200)
```

### **local.settings.json**

---

Ce fichier n'est pas publié lors d'un déploiement :
- YOUR_NAME est le nom du projet Azure Functions ;
- YOUR_KEY est 


```
{
  "IsEncrypted": false,
  "Values": {
    "AzureWebJobsStorage": "DefaultEndpointsProtocol=https; AccountName=YOUR_NAME;AccountKey=YOUR_KEY;EndpointSuffix=core.windows.net",
    "FUNCTIONS_WORKER_RUNTIME": "python",
    "FUNCTIONS_EXTENSION_VERSION": "~3",
    "APPINSIGHTS_INSTRUMENTATIONKEY": "YOUR_KEY"
  },
  "Host": {
    "LocalHttpPort": 7071,
    "CORS": "*",
    "CORSCredentials": false
  }
}
```



### **App.js de l'application mobile Bookshelf**

---

Correction de la ligne 61 : code d'origine

```
59          <FlatList
60            style={{ maxHeight: 200 }}
61            data={this.state.recommendations.map(key => ({
62              key: key.toString()
63            }))}
64            renderItem={({ item }) => <Text>Article n°{item.key}</Text>}
65          />
```

Code corrigé

```
59          <FlatList
60            style={{ maxHeight: 200 }}
61            data={this.state.recommendations.split(',').map(key => ({
62              key: key.toString()
63            }))}
64            renderItem={({ item }) => <Text>Article n°{item.key}</Text>}
65          />
```




### **config.json de l'application mobile Bookshelf**

---
Ici, nous mettons à jour le lien URI pour l’appel en production à l’Azure Functions.

```
{
  "API_URL": "https://YOUR_FUNCTION_NAME.azurewebsites.net/api/YOUR_HTTP_TRIGGER_NAME"
}
```